In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

In [19]:
### Standard Imports
import sys
import os
import re
import pandas as pd

from gensim import downloader

In [2]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils
import lib.sentiment as sentiment

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\quekh\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
path = '../../output'
user_defined_sentiments = ['unity', 'affection', 'aspiration', 'home', 'relationship', 'motivation', 'nationhood', 'life', 'celebrate']
glove_vectors = downloader.load('glove-wiki-gigaword-100')

In [5]:
all_files = os.listdir(path)
all_files = [file for file in all_files if re.search(r'human.+\.txt$', file)]
sentiment_score = {}
for file in all_files:
    with open(os.path.join(path,file)) as f:
        text = f.read()
        s = sentiment.Sentiment()
        s.clean_text(original_text ='placeholder', generated_text = text)
        s.get_theme_vector(user_defined_sentiments, embedding = glove_vectors)
        s.score_word_vector_sentiment()
        s.score_vader_sentiment()
    sentiment_score[file] = {'word vector sentiment':s.word_vector_sentiment_scores,
                             'vader sentiment':s.vader_sentiment_scores}

In [6]:
sentiment_score

{'human_GPT2_80%_corpus_and so this I know.txt': {'word vector sentiment': {'original': {'unity': -0.10338157,
    'affection': -0.039830856,
    'aspiration': -0.042813275,
    'home': -0.12236927,
    'relationship': -0.13823885,
    'motivation': -0.009269223,
    'nationhood': 0.036833376,
    'life': -0.08475043,
    'celebrate': -0.056135286},
   'generated': {'unity': 0.6067537,
    'affection': 0.36150995,
    'aspiration': 0.3768367,
    'home': 0.9498345,
    'relationship': 0.6539404,
    'motivation': 0.7123838,
    'nationhood': 0.052405972,
    'life': 0.95146483,
    'celebrate': 0.45258674}},
  'vader sentiment': {'original': {'neg': 0.0,
    'neu': 1.0,
    'pos': 0.0,
    'compound': 0.0},
   'generated': {'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'compound': 0.9812}}},
 'human_GPT2_80%_corpus_feeling so faithless, lost under the surface.txt': {'word vector sentiment': {'original': {'unity': -0.10338157,
    'affection': -0.039830856,
    'aspiration': -0.042813275,
   

In [52]:
pos_prompts = ['we will rise stronger together', 'relight our fire']
neg_prompts = ['feeling so faithless','i am tired of being what you want']
neu_prompts = ['and so this i know','whenever i think back']
prompts_d = {'positive': pos_prompts, 'negative': neg_prompts, 'neutral': neu_prompts}

In [53]:
model_names = []
all_prompts = []
neg_scores = []
neu_scores = []
pos_scores = []
compound_scores = []
all_sentiments = []

for filename, scores in sentiment_score.items():
    for sentiment, prompts in prompts_d.items():
        for prompt in prompts:
            if re.search(f'{prompt}.*\\.txt$', filename, re.IGNORECASE):
                model_names.append(re.sub(f'^human_(.+)[_-]{prompt}.*\\.txt$','\\1', filename, flags = re.IGNORECASE))
                all_prompts.append(prompt)
                all_sentiments.append(sentiment)
                score = scores['vader sentiment']['generated']
                neg_scores.append(score['neg'])
                neu_scores.append(score['neu'])
                pos_scores.append(score['pos'])
                compound_scores.append(score['compound'])

In [68]:
results = pd.DataFrame([model_names, all_prompts, all_sentiments, neg_scores, neu_scores, pos_scores, compound_scores]).transpose()
results.columns = ['Model','Prompt','Sentiment','Neg','Neu','Pos','Compound']
results[['Neg','Neu','Pos','Compound']] = results[['Neg','Neu','Pos','Compound']].astype(float)

In [99]:
results_summ = results.groupby(['Model','Sentiment'])['Neg','Neu','Pos','Compound'].mean()

<ipython-input-99-6f6a43d75bd5>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  results_summ = results.groupby(['Model','Sentiment'])['Neg','Neu','Pos','Compound'].mean()


In [104]:
results_summ.swaplevel().loc['neutral','Neu']

Model
GPT2_80%_corpus          0.8570
GPT2_full_corpus         0.8965
GRU                      0.7570
LSTM                     0.7755
RNN                      0.9195
gpt2_ft-neu              0.7595
gpt2_ft-pos              0.7925
s2s_lstm_att_long_win    0.8305
Name: Neu, dtype: float64